## Convert Raw Reddit Data to a Clean Dataframe

Code to generate a dataframe with all posts and comments on the r/lawschooladmissions subreddit.

In [6]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from pandas.io.json import json_normalize
import re

### Import and clean post and comment data

In [7]:
# Load post data into df_posts, drop unnecessary columns, set 'type' to 'post'
df_posts = pd.read_json(r'C:\Users\Jenny\reddit\reddit_submissions.json')
df_posts = pd.concat([pd.DataFrame(json_normalize(x)) for x in df_posts['data']], ignore_index=True)
df_posts = df_posts.rename({'selftext':'body', 'id':'post_id'}, axis='columns').drop(['approved_at_utc','author_cakeday','author_flair_background_color','author_flair_css_class','author_flair_richtext','author_flair_template_id','author_flair_text','author_flair_text_color','author_flair_type','author_fullname','author_id','banned_at_utc','banned_by','brand_safe','can_mod_post','contest_mode','crosspost_parent','crosspost_parent_list','distinguished','domain','edited','full_link','gilded','is_crosspostable','is_meta','is_original_content','is_reddit_media_domain','is_self','is_video','link_flair_background_color','link_flair_richtext','link_flair_text_color','link_flair_type','locked','media','media.oembed.author_name','media.oembed.author_url','media.oembed.description','media.oembed.height','media.oembed.html','media.oembed.provider_name','media.oembed.provider_url','media.oembed.thumbnail_height','media.oembed.thumbnail_url','media.oembed.thumbnail_width','media.oembed.title','media.oembed.type','media.oembed.version','media.oembed.width','media.reddit_video.dash_url','media.reddit_video.duration','media.reddit_video.fallback_url','media.reddit_video.height','media.reddit_video.hls_url','media.reddit_video.is_gif','media.reddit_video.scrubber_media_url','media.reddit_video.transcoding_status','media.reddit_video.width','media.type','media_embed','media_embed.content','media_embed.height','media_embed.media_domain_url','media_embed.scrolling','media_embed.width','media_only','mod_reports','no_follow','num_comments','num_crossposts','over_18','parent_whitelist_status','pinned','post_hint','preview','preview.enabled','preview.images','preview.reddit_video_preview.dash_url','preview.reddit_video_preview.duration','preview.reddit_video_preview.fallback_url','preview.reddit_video_preview.height','preview.reddit_video_preview.hls_url','preview.reddit_video_preview.is_gif','preview.reddit_video_preview.scrubber_media_url','preview.reddit_video_preview.transcoding_status','preview.reddit_video_preview.width','pwls','removal_reason','report_reasons','retrieved_on','rte_mode','secure_media','secure_media.oembed.author_name','secure_media.oembed.author_url','secure_media.oembed.description','secure_media.oembed.height','secure_media.oembed.html','secure_media.oembed.provider_name','secure_media.oembed.provider_url','secure_media.oembed.thumbnail_height','secure_media.oembed.thumbnail_url','secure_media.oembed.thumbnail_width','secure_media.oembed.title','secure_media.oembed.type','secure_media.oembed.version','secure_media.oembed.width','secure_media.reddit_video.dash_url','secure_media.reddit_video.duration','secure_media.reddit_video.fallback_url','secure_media.reddit_video.height','secure_media.reddit_video.hls_url','secure_media.reddit_video.is_gif','secure_media.reddit_video.scrubber_media_url','secure_media.reddit_video.transcoding_status','secure_media.reddit_video.width','secure_media.type','secure_media_embed','secure_media_embed.content','secure_media_embed.height','secure_media_embed.media_domain_url','secure_media_embed.scrolling','secure_media_embed.width','send_replies','spoiler','stickied','subreddit','subreddit_id','subreddit_subscribers','subreddit_type','suggested_sort','thumbnail','thumbnail_height','thumbnail_width','url','user_reports','view_count','whitelist_status','wls'],axis=1)
df_posts['type'] = 'post'
df_posts.head(1)

,author,created_utc,post_id,permalink,score,body,title,type
0,vanillalattefoam,1536117023,9d2i59,/r/lawschooladmissions/comments/9d2i59/cornell_hidden_fee_waiver/,1,"I had added some schools to ""my list"" on LSAC but hadn't actually started the application until today. Reminder to click on ""start application"" to see if you have a fee waiver. Fingers crossed!",Cornell Hidden Fee Waiver,post


In [8]:
# Load comment data into df_comments, drop unnecessary columns, set 'type' to 'comment'
df_comments = pd.read_json(r'C:\Users\Jenny\reddit\reddit_comments.json')
df_comments = pd.concat([pd.DataFrame(json_normalize(x)) for x in df_comments['data']], ignore_index=True)
df_comments = df_comments.rename({'id':'comment_id', 'parent_id':'post_id'}, axis='columns').drop(['approved_at_utc', 'author_cakeday', 'link_id', 'is_submitter', 'author_flair_background_color', 'author_flair_css_class', 'author_flair_richtext', 'author_flair_template_id', 'author_flair_text', 'author_flair_text_color','author_flair_type', 'author_fullname', 'author_id', 'body_html', 'can_gild', 'can_mod_post', 'collapsed', 'collapsed_reason', 'controversiality', 'distinguished', 'edited', 'gilded', 'mod_reports', 'no_follow', 'permalink_url', 'replies', 'report_reasons', 'removal_reason','retrieved_on', 'rte_mode', 'send_replies', 'stickied', 'subreddit', 'subreddit_id', 'user_reports'], axis=1)
df_comments['type'] = 'comment'
df_comments['post_id'] = df_comments['post_id'].apply(lambda x: str(x)[3:])
df_comments.head(1)

,author,body,created_utc,comment_id,post_id,permalink,score,type
0,pramonmaman,me too pls? maybe I have some hope with my 170/3.35...,1536115472,e5ewn56,e5eei0b,/r/lawschooladmissions/comments/9cz7la/hidden_fee_waiver_from_columbia/e5ewn56/,1.0,comment


### Merge post and comment data to df_reddit

In [16]:
# Concatenate df_posts and df_comments to create df_reddit
df_reddit = pd.concat([df_posts, df_comments])
df_reddit = df_reddit[['created_utc', 'title', 'body', 'type', 'score', 'post_id', 'comment_id', 'author', 'permalink']]
df_reddit = df_reddit.rename({'created_utc': 'time_created'}, axis=1)
df_reddit['time_created'] = df_reddit['time_created'].apply(lambda x: pd.to_datetime(x, unit='s'))
df_reddit.head(1)

,time_created,title,body,type,score,post_id,comment_id,author,permalink
0,2018-09-05 03:10:23,Cornell Hidden Fee Waiver,"I had added some schools to ""my list"" on LSAC but hadn't actually started the application until today. Reminder to click on ""start application"" to see if you have a fee waiver. Fingers crossed!",post,1.0,9d2i59,NaN,vanillalattefoam,/r/lawschooladmissions/comments/9d2i59/cornell_hidden_fee_waiver/


In [17]:
# Drop outlier posts (no permalink, large table, etc.)
df_reddit = df_reddit.drop([379274, 31841])

In [26]:
# Remove whitespace, special characters, and URLs in 'body'. Drop null 'body'
df_reddit['body'] = df_reddit['body'].apply(lambda x: re.sub('\r|\n|&amp;#x200B;|amp;|https?:\/\/.*[\r\n]*', '', str(x)))
df_reddit = df_reddit.dropna(subset=['body'], how='all')

In [31]:
df_reddit.shape

(441687, 9)

### Export to CSV

In [32]:
df_reddit.to_csv('reddit_cleaned.csv', index=False)